In [1]:
import numpy as np
import pandas as pd
import os
import json
import emoji
import re
import nltk
import spellchecker
import sklearn
from sentence_transformers import SentenceTransformer

In [2]:
current_dir = os.getcwd()
print("Current Working Directory:", current_dir)
parent_dir = os.path.abspath(os.path.join(current_dir, ".."))
os.chdir(parent_dir)
print("New Working Directory:", os.getcwd())

Current Working Directory: /local/scratch/yxie289/mlproject/docs
New Working Directory: /local/scratch/yxie289/mlproject


In [3]:
data = pd.read_csv('./dataset/basic_info/all.csv')
data.head()

,id,image_path,img_text_path,img_url,labels,tweet_url,tweet_text,labels_str,label,img_text_filename,img_text_available,image_filename,image_available,split
0,1057393737372966912,./dataset/img_resized/1057393737372966912.jpg,./dataset/img_txt/1057393737372966912.json,http://pbs.twimg.com/media/Dqyd_GzW4AE2ElU.jpg,"[5, 1, 1]",https://twitter.com/user/status/10573937373729...,Thats what u call a redneck lol https://t.co/...,"['OtherHate', 'Racist', 'Racist']",1,1057393737372966912.json,True,1057393737372966912.jpg,True,train
1,1057149180148240384,./dataset/img_resized/1057149180148240384.jpg,./dataset/img_txt/1057149180148240384.json,http://pbs.twimg.com/media/Dqu_kg0UwAA70sS.jpg,"[0, 3, 1]",https://twitter.com/user/status/10571491801482...,@mortals @Ghostzro Nigga forgot mortal 😭 morta...,"['NotHate', 'Homophobe', 'Racist']",1,1057149180148240384.json,True,1057149180148240384.jpg,True,train
2,1113184475658293251,./dataset/img_resized/1113184475658293251.jpg,./dataset/img_txt/1113184475658293251.json,http://pbs.twimg.com/media/D3LTXs3W4AAbHrk.jpg,"[1, 0, 1]",https://twitter.com/user/status/11131844756582...,Nigga if ain't interested in playing for the t...,"['Racist', 'NotHate', 'Racist']",1,1113184475658293251.json,True,1113184475658293251.jpg,True,train
3,1108665597510991873,./dataset/img_resized/1108665597510991873.jpg,./dataset/img_txt/1108665597510991873.json,http://pbs.twimg.com/media/D2LFd5WXgAAeK0A.jpg,"[0, 2, 2]",https://twitter.com/user/status/11086655975109...,Love a BBC So faggot slut https://t.co/Woji3I...,"['NotHate', 'Sexist', 'Sexist']",1,1108665597510991873.json,False,1108665597510991873.jpg,True,train
4,1108732814202228737,./dataset/img_resized/1108732814202228737.jpg,./dataset/img_txt/1108732814202228737.json,http://pbs.twimg.com/media/D2MCnJtXgAEKsVc.jpg,"[2, 0, 1]",https://twitter.com/user/status/11087328142022...,@Chief30k leaked Pics of Dog Nigga live action...,"['Sexist', 'NotHate', 'Racist']",1,1108732814202228737.json,False,1108732814202228737.jpg,True,train


In [4]:
def read_img_text(path):
    # read the image-text data for certain samples using the path of a json file
    if path != '':
        with open(path) as json_file:
            data = json.load(json_file)
            return data['img_text']
    return ''
url_pattern = re.compile(r'https?://\S+|www\.\S+')
at_pattern = re.compile(r'@[\w]+')
stop_words = set(nltk.corpus.stopwords.words('english'))
# stemmer = nltk.stem.PorterStemmer()
lemmatizer = nltk.stem.WordNetLemmatizer()
# spell = spellchecker.SpellChecker()
all_texts = []
for i in range(len(data)):
    tweet_text = data.loc[i, 'tweet_text']
    if data.loc[i, 'img_text_available']:
        img_text = read_img_text(data.loc[i, 'img_text_path'])
    else:
        img_text = ''
    all_text = tweet_text + '. ' +  img_text
    all_text = all_text.lower()    # lower all the characters
    all_text = emoji.demojize(all_text)  # translate the emoji
    all_text = url_pattern.sub('', all_text)  # delete the url link
    all_text = at_pattern.sub('', all_text)  # delete the @user mark
    all_text = re.sub(r'[^\w\s]', ' ', all_text)  # delete the punctuation marks
    all_text = re.sub(r'\d+', '', all_text)  # delete the numbers 
    all_texts.append(all_text)
all_texts = np.array(all_texts, dtype=object)

In [19]:
all_embeddings = []
text_encoder = SentenceTransformer('all-mpnet-base-v2')
for idx in range(0, len(all_texts), 10000):
    if idx + 10000 <= len(all_texts):
        text = all_texts[idx: idx+10000]
    else:
        text = all_texts[idx:] 
    embedding = text_encoder.encode(text)
    all_embeddings.append(embedding)
    print(len(all_embeddings))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [24]:
all_embeddings = list(all_embeddings)

In [25]:
np_embeddings = np.concatenate(all_embeddings, axis=0)

In [27]:
splits = np.array(data['split'], dtype='object')
labels = np.array(data['label'])
image_path = np.array(data['image_path'], dtype='object')
np.savez('./dataset/embedding/sentence_bert/sentence_bert_768.npz', embeddings=np_embeddings, splits=splits, labels=labels, image_paths = image_path, allow_pickle=True)

In [28]:
from sklearn.decomposition import PCA

# 使用 PCA 进行降维
pca = PCA(n_components=64)
transformer = pca.fit(np_embeddings[data[data['split'] == 'train'].index])
embeddings_reduced = transformer.transform(np_embeddings)
print(np.cumsum(pca.explained_variance_ratio_))

array([0.05807916, 0.08466152, 0.1080174 , 0.12869711, 0.14777718,
       0.1658229 , 0.18082525, 0.19516644, 0.20879532, 0.22136357,
       0.23348341, 0.24443649, 0.25523715, 0.26582969, 0.27577301,
       0.28509359, 0.2942689 , 0.30299923, 0.31139666, 0.3196926 ,
       0.32757743, 0.33524741, 0.34263299, 0.34979567, 0.35680677,
       0.36371348, 0.37034198, 0.37688621, 0.38323917, 0.38950456,
       0.39555205, 0.40147365, 0.40717143, 0.41280782, 0.41839027,
       0.42382128, 0.42915432, 0.43447537, 0.43963213, 0.4446949 ,
       0.4496177 , 0.45441486, 0.45913956, 0.4638537 , 0.46842809,
       0.47289361, 0.47734925, 0.48172793, 0.48608586, 0.49038069,
       0.494587  , 0.49874938, 0.50285529, 0.50691282, 0.51092944,
       0.51492029, 0.51884587, 0.52271407, 0.52649595, 0.53021249,
       0.53386512, 0.5374677 , 0.54100015, 0.54451039])

In [31]:
splits = np.array(data['split'], dtype='object')
labels = np.array(data['label'])
image_path = np.array(data['image_path'], dtype='object')
np.savez('./dataset/embedding/sentence_bert/sentence_bert_pca_64.npz', embeddings=embeddings_reduced, splits=splits, labels=labels, image_paths = image_path, allow_pickle=True)